In [1]:
import numpy as np
import plotly.graph_objects as go

# 漂移率
k = 0.3       
# 噪声标准差
sigma = 1.0      
B = 2.0     
dt = 0.001   
max_t = 5.0  
max_t_steps = int(max_t / dt) 

In [2]:
def simulate_ddm(k, sigma, B, dt, max_t):
    x = 0.0  # 初始决策变量
    max_t_steps = int(max_t / dt)  
    xs = np.zeros(max_t_steps)  # 存储证据轨迹
    r = 0  
    t_step = 0  

    while abs(x) < B and t_step < max_t_steps - 1:
        # Euler-Maruyama 方法
        dx = k * dt + sigma * np.random.randn() * np.sqrt(dt) 
        x += dx
        xs[t_step] = x
        t_step += 1

    if x >= B:
        r = 1  
        # 剩余时间的值填充为上边界
        xs[t_step:] = B  
    elif x <= -B:
        r = -1  
        # 剩余时间的值填充为下边界
        xs[t_step:] = -B  

    # 返回决策时间、证据轨迹和决策结果
    t_decision = t_step * dt
    return t_decision, xs, r

In [3]:
n_trials = 10000
decisions = np.zeros(n_trials, dtype=int)
# 正确决策的反应时间
rts_correct = []  
# 错误决策的反应时间
rts_error = []  
n_correct = 0  
n_error = 0  
# 平均轨迹（正确决策）
trace_correct = np.zeros(max_t_steps) 
# 平均轨迹（错误决策） 
trace_error = np.zeros(max_t_steps)  

for i in range(n_trials):
    # 模拟单次试验
    rt, xs, decision = simulate_ddm(k, sigma, B, dt, max_t)  
    decisions[i] = decision  
    if decision == 1:  
        rts_correct.append(rt)
        n_correct += 1
        trace_correct = ((n_correct - 1) * trace_correct + xs) / n_correct
    elif decision == -1:  
        rts_error.append(rt)
        n_error += 1
        trace_error = ((n_error - 1) * trace_error + xs) / n_error

In [8]:
from plotly.subplots import make_subplots

# 正确决策反应时间的直方图
histogram_correct = go.Histogram(
    x=rts_correct,
    nbinsx=20,
    name="Correct",
    marker=dict(color="blue"),
    opacity=0.8
)

# 错误决策反应时间的直方图
histogram_error = go.Histogram(
    x=rts_error,
    nbinsx=20,
    name="Error",
    marker=dict(color="red"),
    opacity=0.8
)

layout = go.Layout(
    title="Reaction Time Distribution",
    xaxis=dict(title="Time (s)"),
    yaxis=dict(title="Frequency"),
    barmode="group",  # 将柱状图分组排列
    bargap=0.2        # 设置柱间距
)
fig = go.Figure(data=[histogram_correct, histogram_error], layout=layout)
fig.show()

# 正确和错误试验的漂移轨迹
trajectory_correct = go.Scatter(
    x=np.arange(0, max_t_steps * dt, dt),
    y=trace_correct,
    mode="lines",
    name="Correct Trials",
    line=dict(color="blue")
)
trajectory_error = go.Scatter(
    x=np.arange(0, max_t_steps * dt, dt),
    y=trace_error,
    mode="lines",
    name="Error Trials",
    line=dict(color="red")
)
upper_bound = go.Scatter(
    x=[0, max_t_steps * dt],
    y=[B, B],
    mode="lines",
    name="Upper Bound",
    line=dict(dash="dash", color="green")
)
lower_bound = go.Scatter(
    x=[0, max_t_steps * dt],
    y=[-B, -B],
    mode="lines",
    name="Lower Bound",
    line=dict(dash="dash", color="green")
)
layout_traj = go.Layout(
    title="Average DDM Trajectory",
    xaxis=dict(title="Time (s)"),
    yaxis=dict(title="x (Evidence)"),
    showlegend=True
)
fig_traj = go.Figure(data=[trajectory_correct, trajectory_error, upper_bound, lower_bound], layout=layout_traj)
fig_traj.show()

# 计算准确率和平均反应时间
accuracy = n_correct / (n_correct + n_error)  
mean_rt_correct = np.mean(rts_correct) if rts_correct else None
mean_rt_error = np.mean(rts_error) if rts_error else None
print(f"Accuracy: {accuracy:.4f}")
print(f"Mean RT (correct): {mean_rt_correct:.4f} s")
print(f"Mean RT (error): {mean_rt_error:.4f} s")

Accuracy: 0.7718
Mean RT (correct): 2.3711 s
Mean RT (error): 2.3134 s


模拟随机点动实验，包括了 6 种一致性条件

In [10]:
B = 3  
drift_rate = 6 
sigma = 2  
time_non_decision = 0.3 
coherences = np.array([-0.512, -0.256, -0.128, -0.064, -0.032, 0, 0.032, 0.064, 0.128, 0.256, 0.512])
n_coherences = len(coherences)
total_trials = 10000 
dt = 0.001 
max_t = 5.0 

choices = [[] for _ in range(n_coherences)] 
rts = [[] for _ in range(n_coherences)]

for trial in range(total_trials):
    accum = 0
    rt = 0
    coh_i = np.random.randint(0, n_coherences)  
    coh = coherences[coh_i]
    rt, xs, decision = simulate_ddm(drift_rate * coh, sigma, B, dt, max_t * 2)

    if decision != 0: 
        choices[coh_i].append(decision / 2 + 0.5)  
        rts[coh_i].append(rt + time_non_decision) 
p_right = []
mean_rt = []

for coh_i in range(n_coherences):
    if len(choices[coh_i]) > 0:  
        p_right.append(np.mean(choices[coh_i]))  
        mean_rt.append(np.mean(rts[coh_i])) 
    else:
        p_right.append(-1)  
        mean_rt.append(0)

In [11]:
fig = make_subplots(
    rows=2, cols=1, 
    shared_xaxes=True,  
    vertical_spacing=0.2,  
    subplot_titles=("Psychometric Curve", "Chronometric Curve")
)


psychometric_curve = go.Scatter(
    x=coherences,
    y=p_right,
    mode="lines+markers",
    name="Right Choices",
    line=dict(color="blue"),
    marker=dict(size=8)
)
fig.add_trace(psychometric_curve, row=1, col=1)

chronometric_curve = go.Scatter(
    x=coherences,
    y=mean_rt,
    mode="lines+markers",
    name="Reaction Time",
    line=dict(color="red"),
    marker=dict(size=8)
)
fig.add_trace(chronometric_curve, row=2, col=1)


fig.update_layout(
    title="Psychometric and Chronometric Curves",
    xaxis=dict(title="Coherences", showgrid=True),
    yaxis=dict(title="Right Choices (%)", showgrid=True),
    xaxis2=dict(title="Coherences", showgrid=True),
    yaxis2=dict(title="Reaction Time (s)", showgrid=True),
    height=700,  
    width=800,  
    showlegend=True
)

fig.show()

实现 collapsing bound 的 DDM 变体

In [12]:
def simulate_ddm_collapsing_bound(k, sigma, B_initial, collapse_rate, dt, max_t):
    """
    模拟具有 collapsing bound 的 DDM 模型
    collapse_rate: 边界收缩速度
    """
    x = 0.0  
    max_t_steps = int(max_t / dt)  
    xs = np.zeros(max_t_steps)  
    r = 0  
    t_step = 0  

    while t_step < max_t_steps:
        # 随着时间推移逐渐收缩边界
        B = max(B_initial - collapse_rate * t_step * dt, 0)  
        dx = k * dt + sigma * np.random.randn() * np.sqrt(dt)
        x += dx
        xs[t_step] = x

        if x >= B:
            r = 1  
            xs[t_step:] = B  
            break
        elif x <= -B:
            r = -1  
            xs[t_step:] = -B  
            break

        t_step += 1
    t_decision = t_step * dt
    return t_decision, xs, r

In [13]:
# 设置参数
k = 0.3  
sigma = 1.0  
B_initial = 2.0  
collapse_rate = 0.4  
dt = 0.001  
max_t = 5.0  
n_trials = 10000

rts_correct = []
rts_error = []
trace_correct = np.zeros(int(max_t / dt))
trace_error = np.zeros(int(max_t / dt))
n_correct = 0
n_error = 0

for _ in range(n_trials):
    rt, xs, decision = simulate_ddm_collapsing_bound(k, sigma, B_initial, collapse_rate, dt, max_t)

    if decision == 1:
        rts_correct.append(rt)
        n_correct += 1
        trace_correct = ((n_correct - 1) * trace_correct + xs) / n_correct
    elif decision == -1:
        rts_error.append(rt)
        n_error += 1
        trace_error = ((n_error - 1) * trace_error + xs) / n_error

In [15]:
t = np.arange(0, max_t, dt)
dynamic_bounds = np.maximum(B_initial - collapse_rate * t, 0)
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=t,
    y=trace_correct,
    mode="lines",
    name="Average Trajectory of Correct Trials",
    line=dict(color="blue")
))
fig.add_trace(go.Scatter(
    x=t,
    y=trace_error,
    mode="lines",
    name="Average Trajectory of Error Trials",
    line=dict(color="red")
))
fig.add_trace(go.Scatter(
    x=t,
    y=dynamic_bounds,
    mode="lines",
    name="Upper Bound (Collapsing)",
    line=dict(color="green", dash="dash")
))

fig.add_trace(go.Scatter(
    x=t,
    y=-dynamic_bounds,
    mode="lines",
    name="Lower Bound (Collapsing)",
    line=dict(color="green", dash="dash")
))

fig.update_layout(
    title="Average DDM Trajectory with Collapsing Bound",
    xaxis=dict(title="Time (s)"),
    yaxis=dict(title="Evidence"),
    height=600
)

fig.show()

accuracy = n_correct / (n_correct + n_error)
mean_rt_correct = np.mean(rts_correct)
mean_rt_error = np.mean(rts_error)
print(f"Accuracy: {accuracy:.4f}")
print(f"Mean RT (correct): {mean_rt_correct:.4f} s")
print(f"Mean RT (error): {mean_rt_error:.4f} s")

Accuracy: 0.6866
Mean RT (correct): 1.6772 s
Mean RT (error): 1.8719 s


从该曲线可以看出，当 DDM 模型具有 collapsing bound 时，平均的反应时间相较于普通的 DDM 模型缩短，但是准确率有所下降。